<a href="https://colab.research.google.com/github/BigTMiami/AdaptOrDie/blob/main/Amazon_Dataset_Creation_Example.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Amazon Data Prep

This shows how to download a file, tokenize it and push it up to huggingface.  I can do all of this on my local system and will do that, but wanted to post this to show how I am doing this.  


This website was my template for doing this.
https://github.com/huggingface/notebooks/blob/main/examples/language_modeling_from_scratch.ipynb

In [1]:
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 4.4 MB/s eta 0:00:00


In [2]:
!curl -L -k -o train.json.gz  https://jmcauley.ucsd.edu/data/amazon_v2/categoryFilesSmall/AMAZON_FASHION_5.json.gz

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  280k  100  280k    0     0   666k      0 --:--:-- --:--:-- --:--:--  667k


In [3]:
import gzip
import shutil
with gzip.open('train.json.gz', 'rb') as f_in:
    with open('train2.json', 'wb') as f_out:
        shutil.copyfileobj(f_in, f_out)

#!cat train2.json

In [4]:
import json
file = "train2.json"  # e.g., "All_Beauty.jsonl", downloaded from the `review` link above

reviews = []

previous_review = ""

with open(file, "r") as fp:
    line_no = 0
    for line in fp:
        line_no += 1
        entry = json.loads(line.strip())
        if "reviewText" not in entry:
            continue
        review = entry["reviewText"].strip()
        review = review.replace("\n", " ")
        review = review.replace("\n", " ")
        if "\n" in review:
            print(f"{line_no}: Still has return")
        if review == previous_review or len(review) == 0 or review is None:
            continue
        previous_review = review
        reviews.append(entry["reviewText"] + "\n")


review_filename = "AMAZON_FASHION_5_REVIEWS.txt"

with open(review_filename, "w") as fp:
    fp.writelines(reviews)


len(reviews)

3136

This loads the dataset up twice as both train and validation.  This is not correct and only done for this example.

In [5]:
from datasets import load_dataset
dataset = load_dataset("text", data_files={"train": review_filename, "validation": review_filename})

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

In [6]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("roberta-base")
tokenizer.model_max_length
block_size = tokenizer.model_max_length
print(f"block_size:{block_size}")

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/481 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

block_size:512


In [7]:
def tokenize_function(examples):
    # We use `return_special_tokens_mask=True` because DataCollatorForLanguageModeling (see below) is more
    # efficient when it receives the `special_tokens_mask`.
    # return tokenizer(examples["text"], return_special_tokens_mask=True)
    return tokenizer(examples["text"])

In [8]:
tokenized_datasets = dataset.map(
    tokenize_function, batched=True, num_proc=4, remove_columns=["text"]
)
tokenized_datasets["train"]

Map (num_proc=4):   0%|          | 0/3504 [00:00<?, ? examples/s]

Map (num_proc=4):   0%|          | 0/3504 [00:00<?, ? examples/s]

Dataset({
    features: ['input_ids', 'attention_mask'],
    num_rows: 3504
})

In [9]:
#Show three examples
for i in range(3):
    print(dataset["train"]["text"][i])
    print(tokenized_datasets["train"]["input_ids"][i])

Great product and price!
[0, 19065, 1152, 8, 425, 328, 2]
Waaay too small. Will use for futur children!
[0, 771, 6621, 857, 350, 650, 4, 2290, 304, 13, 20136, 710, 408, 328, 2]
Stays vibrant after many washes
[0, 5320, 4113, 11577, 71, 171, 21, 5065, 2]


In [10]:
def group_texts(examples):
    # Concatenate all texts.
    concatenated_examples = {k: sum(examples[k], []) for k in examples.keys()}
    total_length = len(concatenated_examples[list(examples.keys())[0]])
    # We drop the small remainder, we could add padding if the model supported it instead of this drop, you can
    # customize this part to your needs.
    total_length = (total_length // block_size) * block_size
    # Split by chunks of max_len.
    result = {
        k: [t[i : i + block_size] for i in range(0, total_length, block_size)]
        for k, t in concatenated_examples.items()
    }
    result["labels"] = result["input_ids"].copy()
    return result

In [11]:
lm_datasets = tokenized_datasets.map(
    group_texts,
    batched=True,
    batch_size=1000,
    num_proc=4,
)

tokenizer.decode(lm_datasets["train"][0]["input_ids"])

Map (num_proc=4):   0%|          | 0/3504 [00:00<?, ? examples/s]

Map (num_proc=4):   0%|          | 0/3504 [00:00<?, ? examples/s]

'<s>Great product and price!</s><s>Waaay too small. Will use for futur children!</s><s>Stays vibrant after many washes</s><s>My son really likes the pink. Ones which I was nervous about</s><s>Waaay too small. Will use for future child.</s><s>Relieved my Plantar Fascitis for 3 Days. Then the unbearable pain returned in full force. These were recommended by my Podiatrist.</s><s>This is my 6th pair and they are the best thing ever for my plantar fasciitis and resultant neuromas. Unfortunately, the ones I ordered from SmartDestination must be seconds as they kill my feet. The hard plastic insert rubs on the outside edges of my feet. I am unable to exchange them as I waited one day too late to use them in my walking shoes.</s><s>We have used these inserts for years.  They provide great support.</s><s>Pinnacle seems to have more cushioning so my husband likes them better. He\'s tried all the other Powerstep inserts as well as other brands. This is his must have insert. He is 6\'3" and weighs

Because I pushed this, it is saved in BigTMiami/amazon_review_example_tokens_only_dataset , but it is publicly available.

In [12]:
lm_datasets.push_to_hub("amazon_review_example_tokens_only_dataset")

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

CommitInfo(commit_url='https://huggingface.co/datasets/BigTMiami/amazon_review_example_tokens_only_dataset/commit/0e8367d5705c52378f6aa48e7bb84495e87aafd3', commit_message='Upload dataset', commit_description='', oid='0e8367d5705c52378f6aa48e7bb84495e87aafd3', pr_url=None, pr_revision=None, pr_num=None)